<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Chocolate Gummy Bears](#Q2)
    - Perceptron
    - Multilayer Perceptron
4. [Keras MMP](#Q3)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** This is what takes the inputs, multiplies them by the weights, and does the sum or whatever calculation is being performed on the data. This is basically what does the work for the hidden layers. 

- **Input Layer:**  This is what takes in the data that is being fed into the model. 

- **Hidden Layer:** This is where the neural network takes the input and processes it into the output given weights and biases.

- **Output Layer:** This is what comes out of the model, either a classification or a prediction value or set of values. 

- **Activation:** This defines the output of the neuron given a particular input. This is what is used to do the calculation. 

- **Backpropagation:** This computs the gradient with respect to the loss function and the weights of the network for a single input-output. This algorithm works recursively. 


## 2. Chocolate Gummy Bears <a id="Q2"></a>

Right now, you're probably thinking, "yuck, who the hell would eat that?". Great question. Your candy company wants to know too. And you thought I was kidding about the [Chocolate Gummy Bears](https://nuts.com/chocolatessweets/gummies/gummy-bears/milk-gummy-bears.html?utm_source=google&utm_medium=cpc&adpos=1o1&gclid=Cj0KCQjwrfvsBRD7ARIsAKuDvMOZrysDku3jGuWaDqf9TrV3x5JLXt1eqnVhN0KM6fMcbA1nod3h8AwaAvWwEALw_wcB). 

Let's assume that a candy company has gone out and collected information on the types of Halloween candy kids ate. Our candy company wants to predict the eating behavior of witches, warlocks, and ghosts -- aka costumed kids. They shared a sample dataset with us. Each row represents a piece of candy that a costumed child was presented with during "trick" or "treat". We know if the candy was `chocolate` (or not chocolate) or `gummy` (or not gummy). Your goal is to predict if the costumed kid `ate` the piece of candy. 

If both chocolate and gummy equal one, you've got a chocolate gummy bear on your hands!?!?!
![Chocolate Gummy Bear](https://ed910ae2d60f0d25bcb8-80550f96b5feb12604f4f720bfefb46d.ssl.cf1.rackcdn.com/3fb630c04435b7b5-2leZuM7_-zoom.jpg)

In [2]:
import pandas as pd
candy = pd.read_csv('chocolate_gummy_bears.csv')

In [3]:
print(candy.shape)
candy.head()

(10000, 3)


,chocolate,gummy,ate
0,0,1,1
1,1,0,1
2,0,1,1
3,0,0,0
4,1,1,0


### Perceptron

To make predictions on the `candy` dataframe. Build and train a Perceptron using numpy. Your target column is `ate` and your features: `chocolate` and `gummy`. Do not do any feature engineering. :P

Once you've trained your model, report your accuracy. You will not be able to achieve more than ~50% with the simple perceptron. Explain why you could not achieve a higher accuracy with the *simple perceptron* architecture, because it's possible to achieve ~95% accuracy on this dataset. Provide your answer in markdown (and *optional* data anlysis code) after your perceptron implementation. 

In [4]:
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Creating sigmoid functions
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
    return 1/(1 - sigmoid(x))

X = candy[['chocolate', 'gummy']].values
y = candy['ate'].values

correct_output = np.array(y)
correct_output

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [5]:
# Perceptron function

np.random.seed(42)
weights = np.random.random((2,1))
print(f'weights:\n {weights}\n')
print(X.shape, weights.shape, '\n')

inputs = X
weighted_sum = np.dot(inputs, weights)
print(f'weighted_sum:\n {weighted_sum}\n')

activated_outputs = sigmoid(weighted_sum)
print(f'activated_outputs:\n {activated_outputs}\n')

error = correct_output - activated_outputs
print(f'error:\n {error}\n')

adjustments = error * sigmoid_derivative(activated_outputs)
print(f'adjustments:\n {adjustments}\n')

weights = weights + np.dot(inputs.T, adjustments)
print(f'weights:\n {weights}\n')

weights:
 [[0.37454012]
 [0.95071431]]

(10000, 2) (2, 1) 

weighted_sum:
 [[0.95071431]
 [0.37454012]
 [0.95071431]
 ...
 [0.95071431]
 [0.95071431]
 [0.37454012]]

activated_outputs:
 [[0.72125881]
 [0.59255557]
 [0.72125881]
 ...
 [0.72125881]
 [0.72125881]
 [0.59255557]]

error:
 [[0.27874119 0.27874119 0.27874119 ... 0.27874119 0.27874119 0.27874119]
 [0.40744443 0.40744443 0.40744443 ... 0.40744443 0.40744443 0.40744443]
 [0.27874119 0.27874119 0.27874119 ... 0.27874119 0.27874119 0.27874119]
 ...
 [0.27874119 0.27874119 0.27874119 ... 0.27874119 0.27874119 0.27874119]
 [0.27874119 0.27874119 0.27874119 ... 0.27874119 0.27874119 0.27874119]
 [0.40744443 0.40744443 0.40744443 ... 0.40744443 0.40744443 0.40744443]]

adjustments:
 [[0.85211767 0.85211767 0.85211767 ... 0.85211767 0.85211767 0.85211767]
 [1.14435026 1.14435026 1.14435026 ... 1.14435026 1.14435026 1.14435026]
 [0.85211767 0.85211767 0.85211767 ... 0.85211767 0.85211767 0.85211767]
 ...
 [0.85211767 0.85211767 0.852117

In [9]:
for iteration in range(100):
    
    # Weighted sum of inputs / weights
    weighted_sum = np.dot(inputs, weights)
    
    # Activate!
    activated_output = sigmoid(weighted_sum)
    
    # Cac error
    error = correct_output - activated_output
    
    adjustments = error * sigmoid_derivative(activated_output)
    
    # Update the Weights
    weights += np.dot(inputs.T, adjustments)
    
print("Weights after training")
print(weights)

print("Output after training")
print(activated_output)

C:\Users\Rob\.virtualenvs\DS-Unit-4-Sprint-2-Neural-Networks-AITeJwQM\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  
Weights after training
[[4531.41663252 4531.41663252 4531.41663252 ... 4531.41663252
  4531.41663252 4531.41663252]
 [3806.33011798 3806.33011798 3806.33011798 ... 3806.33011798
  3806.33011798 3806.33011798]]
Output after training
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


### Multilayer Perceptron <a id="Q3"></a>

Using the sample candy dataset, implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. Your Multilayer Perceptron should be implemented in Numpy. 
Your network must have one hidden layer.

Once you've trained your model, report your accuracy. Explain why your MLP's performance is considerably better than your simple perceptron's on the candy dataset. 

In [12]:
X.shape, y.shape

((10000, 2), (10000,))

In [29]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = X.shape[1]
        self.hiddenNodes = 3
        self.outputNodes = 1

        # Initial Weights
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)

        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [30]:
nn = NeuralNetwork()

# Number of Epochs / Iterations
for i in range(10):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[0 1]
 [1 0]
 [0 1]
 ...
 [0 1]
 [0 1]
 [1 0]]
Actual Output: 
 [1 1 1 ... 1 1 1]
Predicted Output: 
 [[0.78954815]
 [0.7849666 ]
 [0.78954815]
 ...
 [0.78954815]
 [0.78954815]
 [0.7849666 ]]
Loss: 
 0.33217989658983954


ValueError: shapes (10000,10000) and (1,3) not aligned: 10000 (dim 1) != 1 (dim 0)

In [31]:
# nn = NeuralNetwork()
# nn.train(X, y)

P.S. Don't try candy gummy bears. They're disgusting. 

## 3. Keras MMP <a id="Q3"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV or RandomSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 3 parameters in order to get a 3 on this section.

In [32]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
71,51,1,2,94,227,0,1,154,1,0.00000,2,1,3,1
44,39,1,2,140,321,0,0,182,0,0.00000,2,0,2,1
87,46,1,1,101,197,1,1,156,0,0.00000,2,0,3,1
13,64,1,3,110,211,0,0,144,1,1.80000,1,0,2,1
41,48,1,1,130,245,0,0,180,0,0.20000,1,0,2,1


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Setting target and features
target = 'target'

# Dropping 'result' and 'age' as they seem to be confounding and not helpful
features = df.columns.drop([target])

# Setting X and y
X = df[features]
y = df[target]

# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Checking data
X_train[0]

(242, 13) (242,) (61, 13) (61,)


array([-0.3896464 ,  0.67015058, -0.9200267 ,  0.44004645,  1.0187346 ,
       -0.43831293,  0.88510589, -1.17558097,  1.46385011,  2.66494207,
       -0.62579811,  2.07087229,  1.10598945])

In [39]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(1, input_shape=[X_train.shape[1]], activation='relu'))
model.compile(
    loss='mse', 
    optimizer='adam', 
    metrics=[['accuracy', 'mean_squared_error']]
    )
history = model.fit(X_train, y_train, validation_split=0.2, epochs=200, verbose=1)
scores = model.evaluate(X_train, y_train)

mean_squared_error: 0.3293
Epoch 107/200
193/193 [==============================] - 0s 83us/sample - loss: 0.3995 - accuracy: 0.5751 - mean_squared_error: 0.3995 - val_loss: 0.3279 - val_accuracy: 0.6735 - val_mean_squared_error: 0.3279
Epoch 108/200
193/193 [==============================] - 0s 83us/sample - loss: 0.3985 - accuracy: 0.5751 - mean_squared_error: 0.3985 - val_loss: 0.3264 - val_accuracy: 0.6735 - val_mean_squared_error: 0.3264
Epoch 109/200
193/193 [==============================] - 0s 83us/sample - loss: 0.3975 - accuracy: 0.5803 - mean_squared_error: 0.3975 - val_loss: 0.3244 - val_accuracy: 0.6735 - val_mean_squared_error: 0.3244
Epoch 110/200
193/193 [==============================] - 0s 73us/sample - loss: 0.3967 - accuracy: 0.5803 - mean_squared_error: 0.3967 - val_loss: 0.3228 - val_accuracy: 0.6735 - val_mean_squared_error: 0.3228
Epoch 111/200
193/193 [==============================] - 0s 73us/sample - loss: 0.3955 - accuracy: 0.5803 - mean_squared_error: 0.395

### The baseline model above gives man an accuracy of roughly 69-70% at its best point around epoch 200. 

### Hyperparameter tuning below:

In [57]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(18, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='relu'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [30],
              'epochs': [20, 40, 60, 100]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7392857193946838 using {'batch_size': 30, 'epochs': 60}
Means: 0.7107993364334106, Stdev: 0.088437168915337 with: {'batch_size': 30, 'epochs': 20}
Means: 0.644727885723114, Stdev: 0.08062130482307564 with: {'batch_size': 30, 'epochs': 40}
Means: 0.7392857193946838, Stdev: 0.09834520018201998 with: {'batch_size': 30, 'epochs': 60}
Means: 0.7312074899673462, Stdev: 0.10667494865784993 with: {'batch_size': 30, 'epochs': 100}
